In [ ]:
import numpy as np
import pandas as pd

from sgp4.ext import rv2coe
from sgp4.earth_gravity import wgs72

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# load orbits
# catalog = pd.read_csv('./initial_catalog.csv')
catalog = pd.read_csv('./100k_random_orbits.csv.gz')

catalog = catalog.iloc[0:10000]  # <------------------ limit the object count

catalog.head(2)
if 'placeholder' in catalog.columns: catalog.drop(columns='placeholder',inplace=True)
catalog['size_m'] = np.random.uniform( 3,30,size=(catalog.shape[0]) )
catalog

In [ ]:
# ----- rv2coe ------
# %  outputs       :
# %    p           - semilatus rectum               km
# %    a           - semimajor axis                 km
# %    ecc         - eccentricity
# %    incl        - inclination                    0.0  to pi rad
# %    omega       - longitude of ascending node    0.0  to 2pi rad
# %    argp        - argument of perigee            0.0  to 2pi rad
# %    nu          - true anomaly                   0.0  to 2pi rad
# %    m           - mean anomaly                   0.0  to 2pi rad
# %    arglat      - argument of latitude      (ci) 0.0  to 2pi rad
# %    truelon     - true longitude            (ce) 0.0  to 2pi rad
# %    lonper      - longitude of periapsis    (ee) 0.0  to 2pi rad
    
coe = catalog.apply( lambda X: 
                    rv2coe( X[ ['gcrsx','gcrsy','gcrsz']],
                            X[ ['gcrsdx','gcrsdy','gcrsdz']],
                            wgs72.mu ),
          axis=1 ).values
coe = np.vstack( coe )

# add these to frame
for i,name in enumerate( ['p','a','ecc','incl','omega','argp','nu','m','arglat','truelon','lonper'] ):
    catalog[ name ] = coe[:,i]

catalog['apogee'] = catalog.apply( lambda R: R['a'] * (1 + R['ecc'] ) , axis=1)
catalog['perigee'] = catalog.apply( lambda R: R['a'] * (1 - R['ecc'] ) , axis=1)

In [ ]:
catalog

In [ ]:
leo = catalog[ catalog['perigee'] < 7500 ]
leod = leo.to_dict('records')

# a lookup dict
leo_lookup = { int(X['scc']) : X for X in leod }
len(leod)

## Velocity equations

Radius (equation 4, Matney)
$$
\dot{r}\left(r\right) = \pm \sqrt{ \frac{\mu}{a} } \frac{ \sqrt{ (r_a - r) (r-r_p) } }{r}
$$

Longitude (Equation 12, Matney)
$$
\nu_{\phi} \left(r,\lambda\right) = \frac{1}{r} \sqrt{ \frac{\mu r_A, r_P }{a} }\frac{cos(i)}{cos(\lambda)}
$$

Latitude (Equation 13, Matney)
$$
\nu_{\lambda} \left(r,\lambda\right) = \pm \frac{1}{r} \sqrt{ \frac{\mu r_A, r_P }{a} }  \sqrt{ 1 - \frac{cos(i)}{cos(\lambda)} }
$$

Vis-viva
$$
\begin{align}
v^2 & = & G M  \left( \frac{2}{r} - \frac{1}{a} \right) \\
v & = & \sqrt{ \mu \left( \frac{2}{r} - \frac{1}{a} \right)  }
\end{align}
$$

In [ ]:
def velocity_rad( apogee, perigee, a, r ):
    A = np.sqrt( wgs72.mu / a )
    B = np.sqrt( (apogee - r) * (r - perigee) )/ r
    return A*B

def velocity_lat( apogee, perigee, a, incl, r, lam ):
    A = 1/r
    B = np.sqrt( (wgs72.mu * apogee * perigee / a  ) )
    C = np.sqrt( 1 - np.cos( incl ) / np.cos( lam ) )
    return A * B * C

def velocity_lon( apogee, perigee, a,incl, r, lam ):
    A = 1/r
    B = np.sqrt( (wgs72.mu * apogee * perigee) / a  )
    C = np.cos( incl ) / np.cos( lam )
    return A * B * C

def vis_viva( a, r ):
    return np.sqrt( wgs72.mu * ((2/r) - (1/a)) )

def total_velocity( orbit_row, r, lam  ):
    if r < orbit_row['perigee'] : return np.nan
    if r > orbit_row['apogee']  : return np.nan
    apogee  = orbit_row['apogee']
    perigee = orbit_row['perigee']
    a       = orbit_row['a']
    incl    = orbit_row['incl']
    return np.sqrt( velocity_rad( apogee, perigee, a, r )**2 + \
                   velocity_lat( apogee, perigee, a , incl, r, lam )**2 + \
                   velocity_lon( apogee, perigee, a , incl, r, lam )**2 ) 

ALT = 7400
print( velocity_lat( leod[0]['apogee'], leod[0]['perigee'], leod[0]['a'], leod[0]['incl'], ALT, 0) )
print( velocity_lon( leod[0]['apogee'], leod[0]['perigee'], leod[0]['a'], leod[0]['incl'], ALT, 0) )
print( velocity_rad( leod[0]['apogee'], leod[0]['perigee'], leod[0]['a'], ALT) )
print( total_velocity( leod[0], ALT, 0) )
vis_viva( leod[0]['a'], ALT )




### Spatial Density vs r/lam

Equation 11 Matney
$$
\begin{align}
\rho(r,\lambda,\phi)  & = & \frac{ p(r) dr \, p(\lambda) d\lambda \, p(\phi) d\phi }{d V} \\
& = & \frac{1}{ 2 \pi^3 r a \sqrt{ sin^2(i) - sin^2(\lambda) } } \cdot \frac{1} { \sqrt{ (r_a-r)(r-r_p) } }
\end{align}
$$

Collisions

$$
\begin{align}
C & = & S \cdot V  \cdot \alpha \cdot t \\
C & = & \textrm{collisions} \\
S & = &\textrm{spatial density} \\
V & = &\textrm{velocity} \\ 
\alpha & = &\textrm{area (of object)} \\
t & = & \textrm{time} \\
\end{align}
$$


Units
$$
\textrm{#}  =  \frac{1}{km^3} \cdot \frac{km}{s} \cdot km^2 \cdot s
$$

In [ ]:
def spatial_density_calc( orbit_row, r, lam ):
    A = 1 / (2 * np.pi**3 * r * orbit_row['a'] * np.sqrt( np.sin( orbit_row['incl'] ) ** 2 - np.sin(lam) ** 2))
    B = np.sqrt( 1 / ((orbit_row['apogee'] -r ) * (r - orbit_row['perigee'] ))**2 )
    return A * B

def spatial_density( orbit_row, r, lam):
    if r < orbit_row['perigee'] : return 0
    if r > orbit_row['apogee']  : return 0
    return spatial_density_calc( orbit_row, r, lam )

In [ ]:
def num_coll( R ):
    if R['scc'] not in leo_lookup: return np.nan
    size_m = leo_lookup[ R['scc'] ]['size_m']
    return R['velo'] * R['density'] * size_m / 1e6

outdf['num_coll_s'] = outdf.apply( num_coll, axis=1 )

In [ ]:
outdf['num_coll_yr'] = outdf['num_coll_s'] * (86400 * 365.25)
outdf

In [ ]:
outdf['num_coll_yr'].sum()

### Averaged

Equation 16, Matney

$$
\langle p(r,\lambda,\phi) \rangle = \left[ \frac{ \sqrt{ (r_A-r)(r-r_P) }  } {\pi a }  - \frac{1}{\pi}  arcsin \left( \frac{2(a-r)}{r_A-r_P} \right) \right]^{r_2}_{r_1} 
\cdot \frac{\left[ - \frac{1}{\pi} arcsin \left( \frac{sin(\lambda)}{sin(i)} \right) \right]^{\lambda_2}_{\lambda_1} } 
{ \frac{2\pi}{3} (r_2^3 - r_1^3)( sin(\lambda_2) - sin(\lambda_1) ) }
$$

In [ ]:
# "Algorithms for the computation of debris risk", Matney, NASA Orbital Debris Office
# eq 16

def radius_func( apogee, perigee, a, incl, r ):
    A = np.sqrt( ( apogee - r) * (r - perigee) ) / (np.pi * a)
    B = (1/np.pi) * np.arcsin(  ( 2 * (a -r ) )/ ( apogee - perigee ) )
    return A - B

def numerator_func( incl, lam) :
    return -(1/np.pi) * np.arcsin( np.sin( lam ) / np.sin( incl ) )

def average_spatial_density( orbit_row, r2, r1, lam2, lam1 ):
    apogee  = orbit_row['apogee']
    perigee = orbit_row['perigee']
    if perigee > r2 : return 0
    if apogee  <  r1 : return 0
    a       = orbit_row['a']
    incl    = orbit_row['incl']
    left  = radius_func( apogee, perigee, a, incl, r2 ) - radius_func( apogee, perigee, a, incl, r1 )
    numer = numerator_func( incl, lam2 ) - numerator_func( incl, lam1 )
    den   = ((2 * np.pi) / 3) * (r2**3 -r1**3) * (np.sin( lam2 ) - np.sin(lam1) )
    ans = left * (numer/den)
    if np.isnan(ans) : return 0
    return ans
    
dct = catalog.to_dict('records')
# average_spatial_density( dct[0], 42200, 42100, np.radians(80), np.radians(75) )
radius_func( leod[0]['apogee'], leod[0]['perigee'],leod[0]['a'],leod[0]['incl'], 7150)
average_spatial_density( leod[0], 7140,7130,0,np.radians(5)) * 7 * 10e-9 * 86400 * 365

In [ ]:
lams = [np.radians(X) for X in np.arange(0,90,1)]
alts = np.arange(200,1500,10) + wgs72.radiusearthkm

lam_pairs = list(zip( lams[:-1], lams[1:] ) )
alt_pairs = list(zip( alts[:-1], alts[1:] ) )

print('Lambda intervals: {}, Alt intervals: {}'.format( len(lam_pairs), len(alt_pairs) ) )

out = np.zeros( shape=( len(lam_pairs) * len(alt_pairs) * len(leod), 5 ) )

i = 0
for lam1, lam2 in lam_pairs:
    print(lam1)
    for alt1, alt2 in alt_pairs:
        for sat in leod:
            spat_dens = average_spatial_density( sat, alt2, alt1, lam2, lam1 )
            if spat_dens == 0 or np.isnan( spat_dens ): continue
            alt = (alt2 + alt1) / 2
            lam = (lam2 + lam1) / 2
            velo      = vis_viva( sat, alt  )  # just get the midpoint
            out[i,:] = [sat['scc'], alt, lam, velo, spat_dens]
            i += 1



In [ ]:
outdf = pd.DataFrame( out[:i,:], columns=['scc','alt','lam','velo','density'])
outdf

In [ ]:
outdf['dens']

In [ ]:
def getSize( scc ):
    if scc in leo_lookup: return leo_lookup[scc]['size_m']
    return 0.0

def binCollisions( G ) :
    sizes    =  [getSize(X) for X in G['scc'].values] 
    velo     = G['velo'].mean()
    density  = G['density'].sum()
    size     = np.mean( sizes ) / 1e6
    return velo * density * size

collisions = outdf.groupby(['alt','lam']).apply( binCollisions )

In [ ]:
collisions.index